### Playing with langchain and ollama

In [19]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage

In [3]:
# Using llama2 running locally
llm = Ollama(model='llama2')

In [24]:
%%time
llm.invoke('What is weight decay?')

CPU times: user 62 ms, sys: 13.5 ms, total: 75.6 ms
Wall time: 9.05 s


"\nWeight decay, also known as L2 penalization or regularization, is a technique used in machine learning to prevent overfitting. It involves adding a penalty term to the loss function that is proportional to the magnitude of the model's weights. The purpose of weight decay is to shrink the model's weights towards zero, which can help improve its generalization performance by reducing its sensitivity to the training data.\n\nThe weight decay term is usually added to the loss function in the following form:\n\nL = L(y, y') + α \\* (w)²\n\nwhere:\n\n* L(y, y') is the original loss function, which measures the difference between the predicted output and the true output.\n* α is a hyperparameter that controls the strength of the weight decay penalty.\n* w is the magnitude of the model's weights.\n\nThe effect of weight decay is to increase the cost of having large weights. This discourages the model from using large weights, which can help prevent overfitting. The optimal solution will hav

In [25]:
# Creating a prompt 
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a world class technical documentation writer.'),
    ('user', '{input}')
])

In [26]:
chain = prompt | llm

In [27]:
%%time
chain.invoke({'input': 'what is weight decay?'})

CPU times: user 118 ms, sys: 23.8 ms, total: 141 ms
Wall time: 17.5 s


'As a world-class technical documentation writer, I\'m delighted to explain the concept of weight decay to you! Weight decay, also known as "weight decay acceleration," refers to the gradual loss of mass or weight of a spacecraft or satellite over time due to various factors such as radiation, solar wind, and cosmic rays.\n\nThe term "decay" in this context means that the spacecraft or satellite is gradually losing mass or weight, rather than simply drifting or floating in space. This loss of mass can have significant implications for the spacecraft\'s performance and navigation, as well as its overall structural integrity.\n\nWeight decay can occur through several mechanisms:\n\n1. Radiation: Space is filled with high-energy particles such as cosmic rays and solar wind. These particles can collide with the spacecraft\'s structure or materials, causing them to break down or lose mass over time.\n2. Solar wind: The constant bombardment of the spacecraft by the solar wind (a stream of ch

In [28]:
# Adding parser
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [29]:
%%time
chain.invoke({'input': 'What is weight decay?'})

CPU times: user 86.5 ms, sys: 18.3 ms, total: 105 ms
Wall time: 13.6 s


'\nAh, an inquiring mind! *adjusts glasses* Weight decay, my dear human, is a fascinating topic in the realm of technical documentation. It\'s a phenomenon that occurs when the weight of a document becomes too heavy for its own good. 😅\n\nBut fear not, for I shall enlighten you on this mystical concept. Weight decay, also known as "document rot," can happen to any document, be it a manual, a guide, or even a report. It occurs when the information within the document becomes outdated, obsolete, or simply too voluminous to handle.\n\nThink of it like this: Imagine you have a treasure chest filled with gold coins. The chest is heavy, but its weight is bearable. Now, imagine that over time, the coins inside the chest start to rust and decay. The chest becomes heavier and heavier, making it difficult to carry or even move. That\'s what happens when a document decays – it becomes too heavy to handle, much like the treasure chest filled with rusted coins. 💔\n\nSo, how do we prevent weight dec

That is quite a hilarious output from the model. haha

Loading website relevant to the question. And creating retriever that language model can use.

In [9]:
loader = WebBaseLoader('https://paperswithcode.com/method/weight-decay')

docs = loader.load()

In [11]:
embeddings = OllamaEmbeddings(model='llama2')

In [12]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [4]:
prompt = ChatPromptTemplate.from_template('''Answer the following question based on the context provided only:
<context>
{context}
</context>

Questions: {input}''')

document_chain = create_stuff_documents_chain(llm, prompt)

In [7]:
%%time
document_chain.invoke({
    'input': 'What is weight decay?',
    'context': [Document(page_content='weight decay is not a thing.')]
})

CPU times: user 35.1 ms, sys: 8.71 ms, total: 43.9 ms
Wall time: 2.36 s


'I apologize, but as per the context provided, weight decay is not a real or recognized term in any field. Therefore, I cannot provide an answer to your question.'

In [14]:
retriever = vector.as_retriever()
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retriever_chain.invoke({
    'input': 'What is weight decay?'
})
print(response['answer'])

Weight decay, also known as $L_{2}$ regularization, is a regularization technique used in neural networks to prevent overfitting by adding a penalty term to the loss function based on the $L_{2}$ norm of the weights. The penalty term encourages smaller weights, which in turn helps to reduce the complexity of the model and improve its generalization ability. Weight decay can be incorporated directly into the weight update rule, rather than just implicitly by defining it through to objective function.


In [18]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}'),
    ('user', 'Given the above conversation, generate a search query to look up to get information relevant to the conversation')
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
chat_history = [HumanMessage(content='What is weight decay?'), AIMessage(content='I am not sure that is a thing.')]
retriever_chain.invoke({
    'chat_history': chat_history,
    'input': 'How come?'
})

[Document(page_content="Weight Decay Explained | Papers With Code\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Browse State-of-the-Art\n          \n\n\n Datasets \n\n\nMethods\n\n\n\n            More\n          \n\nNewsletter\nRC2022\n\nAbout\nTrends\n\n                      Portals\n                  \n Libraries \n\n\n\n\n\n\n\n\n\n\nSign In\n\n\n\n\n\n\n\n\n\nSubscribe to the PwC Newsletter\n\n×\n\n\n\n\n\n            Stay informed on the latest trending ML papers with code, research developments, libraries, methods, and datasets.\nRead previous issues\n\n\n\n\n\nSubscribe\n\n\n\n\n\n\n\n\n\n\nJoin the community\n\n×\n\n\n\n        You need to log in to edit.\n        You can create a new account if you don't have one.\n\n\n\n\n\n\n\n\n\nEdit Method\n\n×\n\n\n\n\n\n\n\n                Method Name:*\n\n\n\n\n\n\n\n                Method Full Name:*\n\n\n\n\n\n\n\n                Description with Markdown (optional):\n           

In [28]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Answer the user\'s questions based on the below context:\n\n{context}'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}')
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [35]:
chat_history = [HumanMessage(content='What is weight decay?'), AIMessage(content='I CATEGORICALLY DENY THAT WEIGHT DECAY IS A THING!!!')]
retrieval_chain.invoke({
    'chat_history': chat_history,
    'input': 'Tell me in no more than 5 words'
})

{'chat_history': [HumanMessage(content='What is weight decay?'),
  AIMessage(content='I CATEGORICALLY DENY THAT WEIGHT DECAY IS A THING!!!')],
 'input': 'Tell me in no more than 5 words',
 'context': [Document(page_content="Weight Decay Explained | Papers With Code\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Browse State-of-the-Art\n          \n\n\n Datasets \n\n\nMethods\n\n\n\n            More\n          \n\nNewsletter\nRC2022\n\nAbout\nTrends\n\n                      Portals\n                  \n Libraries \n\n\n\n\n\n\n\n\n\n\nSign In\n\n\n\n\n\n\n\n\n\nSubscribe to the PwC Newsletter\n\n×\n\n\n\n\n\n            Stay informed on the latest trending ML papers with code, research developments, libraries, methods, and datasets.\nRead previous issues\n\n\n\n\n\nSubscribe\n\n\n\n\n\n\n\n\n\n\nJoin the community\n\n×\n\n\n\n        You need to log in to edit.\n        You can create a new account if you don't have one.\n\n\n\n\n\n\